## Step 1 - Preparing data for json file

In [ ]:
#Preparing data to save in a Json File
import pandas as pd
import json

artifacts_df = pd.read_csv(open("../data/artifacts_dataset_raw.csv", encoding='utf8'))
artifacts_df.set_index('id', inplace=True)

for i in artifacts_df.index:

  artifacts_df['td_type'][i] = artifacts_df['td_type'][i].replace(' TD','').lower().replace('defect','').replace('security','').split(',')
  artifacts_df['tdma'][i] = artifacts_df['tdma'][i].lower().split(',')
  artifacts_df['output_info'][i] = str(artifacts_df['output_info'][i]).lower().split(',')
  artifacts_df['output_fmt'][i] = artifacts_df['output_fmt'][i].lower().split(',')
  artifacts_df['input_info'][i] = artifacts_df['input_info'][i].lower().split(',')
  artifacts_df['input_fmt'][i] = artifacts_df['input_fmt'][i].lower().split(',')
  artifacts_df['programming_language'][i] = artifacts_df['programming_language'][i].lower().split(',')
  artifacts_df['license_type'][i] = str(artifacts_df['license_type'][i]).lower().split(',')
  artifacts_df['studies_citing_artifact'][i] = artifacts_df['studies_citing_artifact'][i].split(',')
  artifacts_df['other_comments'][i] = str(artifacts_df['other_comments'][i]).split(',')
  artifacts_df['is-integrated-description'][i] = str(artifacts_df['is-integrated-description'][i]).split('\n')
  artifacts_df['is-integrated-with'][i] = str(artifacts_df['is-integrated-with'][i]).split('\n')
  artifacts_df['is-integrated-using-interface-subtype'][i] = str(artifacts_df['is-integrated-using-interface-subtype'][i]).split('\n')
  artifacts_df['interface-subtype'][i] = str(artifacts_df['interface-subtype'][i]).split('\n')
  artifacts_df['trigger-description'][i] = str(artifacts_df['trigger-description'][i]).split('\n')



int_types_parser = {
 '':'',
 'id':'id',
 'cli (standard input)': 'cli',
 'api' : 'api',
 'api (plugin for github)': 'api',
 'gui (plugin for another automation artifact)': 'gui',
 'gui': 'gui',
 'api (integrates with ci)': 'api',
 'gui (plugin for ide)': 'gui'
}

def to_1D(series):
 return pd.Series([x for _list in series for x in _list])

def add_list_to_dict(dict_variable = None, key = None, items_list = None):
  if key in dict_variable.keys():
    dict_variable[key].extend(items_list)
    dict_variable[key] = list(set(dict_variable[key]))
  else:
    dict_variable[key] = items_list
  return dict_variable

## Step 2 - Recovering studies per artifact

In [ ]:
studies_df = pd.read_csv(open("../data/studies_dataset.csv", encoding='utf8'))

new_studies = []
for j in artifacts_df.index:
    n_list = []
    for k in artifacts_df['studies_citing_artifact'][j]:
        try:
            index = studies_df.loc[studies_df['old_id'] == k].index.tolist()[0]
        except:
            pass
    n_list.append(studies_df.at[index,'id'])
    new_studies.append(n_list)
artifacts_df['studies_citing_artifact'] = new_studies

## Step 3 - Retrieving artifact's first citation year

In [ ]:
artifacts_df['first_citation'] = '-'
for i in artifacts_df.index:
    years = set([studies_df['year'][x] for x in studies_df.index if studies_df['id'][x] in artifacts_df['studies_citing_artifact'][i]])
    artifacts_df['first_citation'][i] = min(years)

## Step 4 - Storing Json file

In [ ]:
#Storing dataset in a JSON file

result = artifacts_df.to_json(orient='index')
json_raw = json.loads(result)
with open('../data/Step1_artifacts.json', 'w') as f:
    json.dump(json_raw,f)